In [1]:
import os
import sys
os.environ['PATH']+=':'+sys.path[1].split('lib')[0]+'bin'

import tempfile
import numpy as np

import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app

from openmmtools import states, mcmc
from openmmtools import testsystems

from yank.multistate import ParallelTemperingSampler, MultiStateReporter, ParallelTemperingAnalyzer

import matplotlib.pyplot as plt



plt.style.use("ggplot")

In [ ]:
testsystem = testsystems.AlanineDipeptideImplicit()

In [ ]:
n_replicas = 8
T_min      = 298.0*unit.kelvin
T_max      = 600.0*unit.kelvin

In [ ]:
reference_state = states.ThermodynamicState(system=testsystem.system, temperature=T_min)

In [ ]:
move = mcmc.GHMCMove(timestep=2.0*unit.femtoseconds, n_steps=50)

In [ ]:
simulation = ParallelTemperingSampler(mcmc_moves=move, number_of_iterations=500)

In [ ]:
storage_path = tempfile.NamedTemporaryFile(delete=False).name + '.nc'
reporter = MultiStateReporter(storage_path, checkpoint_interval=10)

In [ ]:
simulation.create(reference_state, states.SamplerState(testsystem.positions), reporter,
                  min_temperature=T_min, max_temperature=T_max, n_temperatures=n_replicas)

In [ ]:
simulation.run(n_iterations=15000)

In [ ]:
analysis=ParallelTemperingAnalyzer(reporter)

In [ ]:
os.remove(storage_path)

In [ ]:
analysis.generate_mixing_statistics()

In [ ]:
print('Max n interations',analysis.max_n_iterations)
print('n equilibration iterations',analysis.n_equilibration_iterations)
print('n replicas',analysis.n_replicas)
print('n states',analysis.n_states)